In [1]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
import math 


In [2]:
robert_frost_file_path = "../data/robert_frost_small.txt"
edgar_allan_poe_file_path = "../data/edgar_allan_poe_small.txt"

In [3]:
# this function is incomplete, still need to do lemmatization and remove puctuation , stopword.
def preprocessing_word(lst):
    return [x.lower().replace("''", "").replace("''", "") for x in lst]

In [4]:
def file_to_tokenize(file_path):
    '''
    This function purpose is to read file and convert to list of tokenize
    '''
    tokenize_list = []
    f = open(file_path, "r")
    for x in f:
        tokenize_list.append(x.split())
        
    # remove empty array
    tokenize_list = [ x for x in tokenize_list if len(x) != 0]
    
    #preprocessing
    tmp = []
    for x in tokenize_list:
        tmp.append(preprocessing_word(x))
        
    tokenize_list = tmp
        
    return tokenize_list

In [5]:
# read file and convert to tokenize by function file_to_tokenize
robert_frost_tokenize_list = file_to_tokenize(robert_frost_file_path)
edgar_allan_poe_tokenize_list = file_to_tokenize(edgar_allan_poe_file_path)
len(robert_frost_tokenize_list),len(edgar_allan_poe_tokenize_list)

(45, 66)

In [6]:
robert_frost_tokenize_list[:5]

[['two', 'roads', 'diverged', 'in', 'a', 'yellow', 'wood,'],
 ['and', 'sorry', 'i', 'could', 'not', 'travel', 'both'],
 ['and', 'be', 'one', 'traveler,', 'long', 'i', 'stood'],
 ['and', 'looked', 'down', 'one', 'as', 'far', 'as', 'i', 'could'],
 ['to', 'where', 'it', 'bent', 'in', 'the', 'undergrowth;']]

In [7]:
edgar_allan_poe_tokenize_list[:5]

[['lo!', 'death', 'hath', "rear'd", 'himself', 'a', 'throne'],
 ['in', 'a', 'strange', 'city,', 'all', 'alone,'],
 ['far', 'down', 'within', 'the', 'dim', 'west'],
 ['where',
  'the',
  'good,',
  'and',
  'the',
  'bad,',
  'and',
  'the',
  'worst,',
  'and',
  'the',
  'best,'],
 ['have', 'gone', 'to', 'their', 'eternal', 'rest.']]

In [8]:
robert_frost_label_list = list(np.zeros(len(robert_frost_tokenize_list)))
edgar_allan_poe_label_list = list(np.ones(len(edgar_allan_poe_tokenize_list)))

len(robert_frost_label_list),len(edgar_allan_poe_label_list)

(45, 66)

In [9]:
# all_data_x = robert_frost_tokenize_list
#all_data_y= robert_frost_label_list+edgar_allan_poe_label_list

# len(all_data_x),len(all_data_y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(robert_frost_tokenize_list, robert_frost_label_list, test_size=0.2, random_state=42)
len(X_train),len(y_train),len(X_test),len(y_test)

(36, 36, 9, 9)

In [11]:
X_train[:5]

[['and', 'looked', 'down', 'one', 'as', 'far', 'as', 'i', 'could'],
 ['and', 'having', 'perhaps', 'the', 'better', 'claim'],
 ['my', 'little', 'horse', 'must', 'think', 'it', 'queer'],
 ['the', 'woods', 'are', 'lovely,', 'dark', 'and', 'deep,'],
 ['and', 'that', 'has', 'made', 'all', 'the', 'difference.']]

In [12]:
X_test[:5]

[['i', 'hold', 'with', 'those', 'who', 'favor', 'fire.'],
 ['to', 'stop', 'without', 'a', 'farmhouse', 'near'],
 ['between', 'the', 'woods', 'and', 'frozen', 'lake'],
 ['is', 'also', 'great'],
 ['and', 'miles', 'to', 'go', 'before', 'i', 'sleep.']]

In [13]:
def mapping_unique_index(lst):
    unique_word_index_dict = {}
    word_index = 0
    for data in lst:
        tmp = np.asarray(data)
        unique_tmp = np.unique(tmp)
        for word in unique_tmp:
            if not word in unique_word_index_dict:
                unique_word_index_dict[word] = word_index
                word_index += 1
    return unique_word_index_dict,word_index

In [14]:
X_train_unique_word_index_dict,X_train_word_index = mapping_unique_index(X_train)
X_test_unique_word_index_dict,X_test_word_index = mapping_unique_index(X_test)

In [15]:
dict(list(X_train_unique_word_index_dict.items())[0:5])

{'and': 0, 'as': 1, 'could': 2, 'down': 3, 'far': 4}

In [16]:
# Finding a key that found in test set but not in trainset
# in case, want to visualize
X_train_keys_set= set(list(X_train_unique_word_index_dict.keys()))
X_test_keys_set= set(list(X_test_unique_word_index_dict.keys()))

not_found_keys = X_test_keys_set - X_train_keys_set

# create not found key to have unknow_word_key
unknow_word_key = 'unk'
X_train_unique_word_index_dict[unknow_word_key] = X_train_word_index

# add unknow index
X_train_word_index += 1
unknow_word_index =X_train_word_index

X_train_unique_word_index_dict[unknow_word_key],X_train_word_index

(157, 158)

In [17]:
# creating dictionary for mapping from index to word 
X_train_unique_index_word_dict = dict((v,k) for k,v in X_train_unique_word_index_dict.items())
dict(list(X_train_unique_index_word_dict.items())[0:5])

{0: 'and', 1: 'as', 2: 'could', 3: 'down', 4: 'far'}

In [18]:
# unique word list -> M
unique_word_train = list(X_train_unique_index_word_dict.keys())

In [19]:
def covert_word_str_to_int(lst,mapping_dict):
    word_int_list = []
    
    for sentence in lst:
        tmp = [mapping_dict[word] if word in mapping_dict else mapping_dict[unknow_word_key] for word in sentence]
        word_int_list.append(tmp)
        
    return word_int_list

In [20]:
# convert from list of string of word to list of index of word
X_train_int = covert_word_str_to_int(X_train,X_train_unique_word_index_dict)
X_test_int = covert_word_str_to_int(X_test,X_train_unique_word_index_dict)

X_train_int = np.asarray(X_train_int)
X_test_int = np.asarray(X_test_int)

len(X_train_int),len(X_test_int)

(36, 9)

In [21]:
X_train_int[:5]

array([list([0, 6, 3, 7, 1, 4, 1, 5, 2]), list([0, 10, 11, 12, 8, 9]),
       list([17, 15, 13, 16, 19, 14, 18]),
       list([12, 24, 20, 23, 21, 0, 22]),
       list([0, 29, 27, 28, 25, 12, 26])], dtype=object)

In [22]:
X_test_int[:5]

array([list([5, 157, 53, 157, 157, 157, 157]),
       list([41, 157, 157, 30, 157, 157]),
       list([157, 12, 24, 0, 157, 157]), list([80, 157, 157]),
       list([0, 112, 41, 111, 110, 5, 157])], dtype=object)

In [23]:
# create dict of initial states 
def create_initial_state(lst,word_number):
    initial_states_dict = {}
    number_of_sequence = len(lst)
    for x in lst:
        count_x = 0
        for y in lst:
            if x[0] == y[0]:
                count_x += 1
        
        #smoothing by add 1
        initial_states_dict[x[0]]= (count_x+1)/(number_of_sequence+word_number)
        
    # unknow word become starting word
    initial_states_dict[unknow_word_key]    = (1)/(number_of_sequence+word_number)
    return initial_states_dict

In [24]:
X_train_initial_states_dict = create_initial_state(X_train_int,X_train_word_index)
dict(list(X_train_initial_states_dict.items())[0:5])

{0: 0.04639175257731959,
 17: 0.010309278350515464,
 12: 0.020618556701030927,
 35: 0.015463917525773196,
 43: 0.010309278350515464}

In [25]:
def count_consecutive_pair_word(sentences,word_1,word_2,word_number):
    count_word_1 = 0
    count_word_1_to_word_2 = 0
#     print(word_1,word_2,type(word_1),type(word_2))
    
    for sentence in sentences:
#         tmp_arr = sentence
#         found_word_1 = [1 for i, x in enumerate(sentence) if x ==word_1]
#         found_word_1 = np.isin(tmp_arr, [word_1])
#         found_word_2 = np.in1d(tmp_arr, [word_2])
#         found_word_1_word_2 = found_word_1 | found_word_2
        
#         print(tmp_arr)
#         print(found_word_1)
#         print(found_word_2)
#         print(found_word_1_word_2)
#         found_pair_list  = [[s, e] for s, e in zip(found_word_1_word_2, found_word_1_word_2[1:]) if (s==True)and (e==True)]
#         count_word_1_to_word_2 += len(found_pair_list)
#         count_word_1 += sum(found_word_1)
        start_w = 0
        stop_w = len(sentence)-1
        for i,local_word in enumerate(sentence[start_w:stop_w],start_w):
            if local_word == word_1 and sentence[i+1] == word_2:
                count_word_1_to_word_2 += 1
            if local_word == word_1:
                count_word_1 += 1
    
    # smooting by add 1
    return (count_word_1_to_word_2+1)/(count_word_1+word_number)

In [26]:
def create_state_transition_matrix(word_unique_list,sentences,word_number):
    state_transition_matrix = np.zeros((word_number, word_number))
    already_done_compute_word_list = []
    for i,word_1 in enumerate(word_unique_list):
        for j,word_2 in enumerate(word_unique_list):
            key = str(word_1)+"-"+str(word_2)
            if (key in already_done_compute_word_list ):
                continue
                
            prob_pair = count_consecutive_pair_word(sentences,word_1,word_2,word_number)
            state_transition_matrix[i][j] = prob_pair
            already_done_compute_word_list.append(key)
#         break
            
    return state_transition_matrix

In [27]:
import time
start_time = time.time()
state_transition_matrix = create_state_transition_matrix(unique_word_train,X_train_int,X_train_word_index)
print("--- %s seconds ---" % (time.time() - start_time))
state_transition_matrix

--- 3.4658477306365967 seconds ---


array([[0.00584795, 0.00584795, 0.00584795, ..., 0.00584795, 0.00584795,
        0.00584795],
       [0.00617284, 0.00617284, 0.00617284, ..., 0.00617284, 0.00617284,
        0.00617284],
       [0.00628931, 0.00628931, 0.00628931, ..., 0.00628931, 0.00628931,
        0.00628931],
       ...,
       [0.00628931, 0.00628931, 0.00628931, ..., 0.00628931, 0.00628931,
        0.00628931],
       [0.00628931, 0.00628931, 0.00628931, ..., 0.00628931, 0.00628931,
        0.00628931],
       [0.00632911, 0.00632911, 0.00632911, ..., 0.00632911, 0.00632911,
        0.00632911]])

In [28]:
state_transition_matrix.shape

(158, 158)

In [29]:
def predict(test_sentences,initial_states_dict,state_transition_matrix):
    
    prob =[]
    word_max_index = state_transition_matrix.shape[1]
    for sentence in test_sentences:
        initial_start_prob = 0
        if sentence[0] in initial_states_dict:
            initial_start_prob = initial_states_dict[sentence[0]]
        else:
            initial_start_prob = initial_states_dict[unknow_word_key]
            
        # stretch value by log function
        initial_start_prob =math.log(initial_start_prob)
        
        sum_of_prob = 0
        
        start_w = 1
        stop_w = len(sentence)-1
        
        for i,local_word in enumerate(sentence[start_w:stop_w],start_w):
            if local_word < 0 or local_word > word_max_index or  sentence[i+1] < 0 or sentence[i+1] > word_max_index:
                sum_of_prob = -100
                break
            sum_of_prob += math.log(state_transition_matrix[local_word][sentence[i+1]])
        
        local_prob = initial_start_prob+sum_of_prob
        prob.append(local_prob)
        
    return prob

In [30]:
predicted_value = predict(X_train_int,X_train_initial_states_dict,state_transition_matrix)
predicted_value

[-33.406346566732324,
 -20.640555903198283,
 -26.47226586258264,
 -25.851878036305123,
 -25.0288130876228,
 -29.342345614958187,
 -30.866294380145447,
 -21.409592210113047,
 -30.167337286944495,
 -26.520390321634842,
 -17.787280623793862,
 -12.96422230208399,
 -22.150498419426956,
 -21.463986282178844,
 -24.89382923901547,
 -21.45148611941461,
 -25.791618844786925,
 -30.534336114224867,
 -26.465996249569045,
 -21.770309741419553,
 -24.61121770763681,
 -26.122572738669692,
 -26.557622160252407,
 -25.02918297850921,
 -25.04745447441173,
 -30.167375866447216,
 -30.540414236568758,
 -20.586161831132486,
 -26.06057020472384,
 -25.44054780874456,
 -7.446390603387394,
 -30.976281522662205,
 -26.53875560385147,
 -30.209600603372223,
 -26.079223274473037,
 -22.09023922790876]

In [31]:
predicted_value = predict(X_test_int,X_train_initial_states_dict,state_transition_matrix)
predicted_value

[-29.20711774528513,
 -24.156945232256724,
 -24.996172370067733,
 -10.330453192090296,
 -25.740601654971673,
 -28.600083578578783,
 -28.63067591329635,
 -35.790244612337794,
 -30.704156531577638]